In [ ]:
!pip install tensorflow 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [47]:
with open("dataset.json") as f:
    data = json.load(f)

X = []
Y = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']).flatten())
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)


In [58]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model


In [54]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [64]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')

In [67]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train), epochs=400)

Epoch 1/400
1/1 [==============================] - 0s 22ms/step - loss: 44.0870
Epoch 2/400
1/1 [==============================] - 0s 20ms/step - loss: 43.6089
Epoch 3/400
1/1 [==============================] - 0s 20ms/step - loss: 43.1586
Epoch 4/400
1/1 [==============================] - 0s 22ms/step - loss: 42.4964
Epoch 5/400
1/1 [==============================] - 0s 23ms/step - loss: 41.9766
Epoch 6/400
1/1 [==============================] - 0s 22ms/step - loss: 41.6436
Epoch 7/400
1/1 [==============================] - 0s 29ms/step - loss: 41.2131
Epoch 8/400
1/1 [==============================] - 0s 26ms/step - loss: 40.8749
Epoch 9/400
1/1 [==============================] - 0s 21ms/step - loss: 40.6158
Epoch 10/400
1/1 [==============================] - 0s 21ms/step - loss: 40.1287
Epoch 11/400
1/1 [==============================] - 0s 24ms/step - loss: 39.7906
Epoch 12/400
1/1 [==============================] - 0s 21ms/step - loss: 39.5613
Epoch 13/400
1/1 [===================

In [66]:
model.fit(x=np.array(X_train),y=np.array(Y_train), epochs=100)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.3868
Epoch 2/100
1/1 [==============================] - 0s 41ms/step - loss: 0.3251
Epoch 3/100
1/1 [==============================] - 0s 43ms/step - loss: 0.2764
Epoch 4/100
1/1 [==============================] - 0s 40ms/step - loss: 0.2314
Epoch 5/100
1/1 [==============================] - 0s 40ms/step - loss: 0.1799
Epoch 6/100
1/1 [==============================] - 0s 41ms/step - loss: 0.1680
Epoch 7/100
1/1 [==============================] - 0s 44ms/step - loss: 0.1327
Epoch 8/100
1/1 [==============================] - 0s 46ms/step - loss: 0.1191
Epoch 9/100
1/1 [==============================] - 0s 53ms/step - loss: 0.1172
Epoch 10/100
1/1 [==============================] - 0s 50ms/step - loss: 0.1048
Epoch 11/100
1/1 [==============================] - 0s 45ms/step - loss: 0.1041
Epoch 12/100
1/1 [==============================] - 0s 43ms/step - loss: 0.0984
Epoch 13/100
1/1 [==============================] -

In [70]:
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))

1/1 [==============================] - 0s 24ms/step


In [71]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
Y_predicted

[array([[0.01638953, 0.01154353, 0.01636174, 0.01816889, 0.04048599,
         0.08941603, 0.04667364, 0.03484448, 0.05066306, 0.02717176,
         0.03850179, 0.05423654, 0.0409501 , 0.14266403],
        [0.01952967, 0.04004342, 0.0342196 , 0.03508776, 0.07929252,
         0.1534603 , 0.06245536, 0.04110529, 0.06104822, 0.05270149,
         0.06918747, 0.08059248, 0.06461877, 0.22254227],
        [0.01828055, 0.02949833, 0.03571711, 0.0357957 , 0.06446811,
         0.13662335, 0.09058513, 0.05464656, 0.06195665, 0.05777768,
         0.0546005 , 0.08407523, 0.0541262 , 0.21745396],
        [0.0260198 , 0.02374825, 0.02381269, 0.13521688, 0.07006123,
         0.18708722, 0.08273142, 0.07284122, 0.09447061, 0.07425327,
         0.13253662, 0.13411441, 0.0960665 , 0.34264058],
        [0.04440967, 0.06303583, 0.07610612, 0.06514592, 0.20538157,
         0.34324363, 0.19588514, 0.10334964, 0.16330707, 0.13247408,
         0.14793596, 0.20161188, 0.13275348, 0.54298294],
        [0.09335422,

In [72]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
Y_ann_predicted

[array([[ 2.5812600e+00, -2.0301774e+00,  4.7203013e-01, -3.7713549e-01,
          1.9527464e+00,  1.5200080e+00,  2.5085394e+00,  2.3184280e+00,
          2.6540914e+00,  9.9135476e-01, -2.0807412e+00, -7.9080391e-01,
          1.5999937e+00, -3.4377199e-01],
        [-2.3493745e+00,  1.0731961e+00,  3.0388626e-01, -1.2178560e+00,
         -6.1550456e-01, -8.2887805e-01, -2.3365702e-01, -9.7405487e-01,
         -1.9400854e+00, -8.4722447e-01,  3.4481094e+00, -5.1359886e-01,
         -5.6218600e-01,  4.6624393e+00],
        [-1.1159036e+00, -1.1623554e+00, -6.0628223e-01, -3.4424903e+00,
         -1.2220461e+00,  2.2039227e+00, -6.6086680e-01, -1.5957694e+00,
         -2.6877327e+00, -2.7001747e-01,  2.0089121e+00, -1.2301604e+00,
          5.4165814e-02, -1.5991125e+00],
        [-3.2170635e-01,  1.0085641e+00,  3.3644364e+00, -1.2053107e+00,
         -4.4460106e-01, -5.3097343e-01, -1.0196310e+00, -2.1096010e+00,
          5.5497211e-01,  9.8929316e-01,  9.4768864e-01, -1.6113625e-01